In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.eval()

Using cache found in C:\Users\marcu/.cache\torch\hub\pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [2]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [3]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([-1.6533e+00, -4.3501e+00, -1.8168e+00, -4.2139e+00, -3.1908e+00,
         3.4174e-01,  1.0880e+00,  5.9346e+00,  8.0415e+00, -7.0259e-01,
        -9.4097e-01, -6.0705e-01, -2.4050e-01, -1.9945e+00, -1.5288e+00,
        -3.2655e+00, -5.5669e-01,  1.0533e+00,  1.9210e-01, -4.7193e+00,
        -3.3874e+00,  4.3047e+00, -1.0996e+00,  4.6139e+00, -5.3750e-03,
        -5.3426e+00, -4.7368e+00, -3.3969e+00, -4.1279e+00,  2.9010e-01,
        -3.2953e+00, -6.7045e+00, -4.7229e+00, -4.1768e+00, -2.1857e+00,
        -2.9464e+00,  3.0467e+00, -3.5876e+00, -6.3874e+00, -4.4200e+00,
        -3.3685e+00, -5.0984e+00, -4.9010e+00, -5.5227e+00, -3.7230e+00,
        -4.0198e+00,  2.6990e-01, -4.4703e+00, -5.6619e+00, -5.4878e+00,
        -2.6803e+00, -3.2127e+00, -1.6294e+00, -5.2283e+00, -2.7478e+00,
        -2.6282e+00, -1.8184e+00, -2.3192e+00, -5.2796e+00, -3.7646e+00,
        -3.0973e+00, -4.1421e+00, -5.2526e+00, -4.6491e+00, -3.5814e+00,
        -4.0182e+00, -4.0006e+00, -4.5504e+00, -3.2

In [ ]:
# use shap to explain predictions of the model on the ImageNet dataset 
